In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from custom_matrix import Matrix, IdentityMatrix
from custom_matrix import simplemul, matmul, transpose, matadd, matsub

m1 = Matrix(3, 4, data=[2, 6, 9, 5, 8, 9, 4, 3, 2, 4, 3, 8])
m2 = Matrix(4, 3, data=[8, 7, 6, 5, 9, 4, 3, 3, 2, 8, 1, 0])
i = IdentityMatrix(3)
print(i.rows)

m = matmul(m1, m2)
print(m.rows)
m = matmul(m, i)
print(m.rows)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[113, 100, 54], [145, 152, 92], [109, 67, 34]]
[[113, 100, 54], [145, 152, 92], [109, 67, 34]]


In [25]:
import pandas as pd
import numpy as np
import fitparse

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go

activity_file = '../../buttes.fit'
# activity_file = '../../canal.fit'
# activity_file = '../../montmartre.fit'
# activity_file = '../../cotes.fit'
# activity_file = '../../seuil.fit'
# activity_file = '../../libourne.fit'
# activity_file = '../../velo.fit'
# activity_file = '../../vma.fit'

fitfile = fitparse.FitFile(activity_file)

session_raw = pd.DataFrame([record.get_values() for record in fitfile.get_messages("record")])
session = session_raw[['timestamp', 'distance', 'enhanced_speed', 'enhanced_altitude', 'heart_rate', 'power']]
start_time = session.timestamp.values[0]
session['time'] = np.arange(session.shape[0])
session.head(20)

,timestamp,distance,enhanced_speed,enhanced_altitude,heart_rate,power,time
0,2022-09-13 10:42:56,0.12,0.000,90.8,67,NaN,0
1,2022-09-13 10:42:57,0.12,0.000,90.8,67,NaN,1
2,2022-09-13 10:42:58,2.24,0.252,90.8,67,NaN,2
3,2022-09-13 10:42:59,4.93,0.392,90.8,66,NaN,3
4,2022-09-13 10:43:00,6.33,1.026,90.8,67,NaN,4
5,2022-09-13 10:43:01,7.86,1.185,90.6,70,NaN,5
6,2022-09-13 10:43:02,10.62,1.381,90.6,71,NaN,6
7,2022-09-13 10:43:03,14.89,1.493,90.6,74,NaN,7
8,2022-09-13 10:43:04,17.09,2.230,90.6,74,NaN,8
9,2022-09-13 10:43:05,18.67,2.771,90.6,74,NaN,9


## 1. Kalman filters

In [26]:
from kalman import KalmanFilter1D
from kalman import KalmanFilter

# Raw speed
# session['speed'] = session.distance.values - session.distance.shift(1).values

# Filtered features
# filtered_speed = KalmanFilter1D(2, 0.5, 1, 0.1)
# session['filtered_speed1'] = [filtered_speed.get_current_estimate(m) for m in session.speed]
# filtered_speed = KalmanFilter1D(2, 0.5, 1, 0.15)
# session['filtered_speed2'] = [filtered_speed.get_current_estimate(m) for m in session.speed]

## BEST ##
# power is already a derived metric, so the 1d is well designed to properly estimate it
# filtered_power_1d = KalmanFilter1D(100, 1, 3, 1)
# session['filtered_power_1d'] = [filtered_power_1d.get_current_estimate(m) for m in session.power]

# filtered_power = KalmanFilter([100, 10], 1, 10, 0.05)
# session['filtered_power'] = [filtered_power.get_current_estimate(m) for m in session.power]

# filtered_altitude = KalmanFilter1D(session.enhanced_altitude[0], 1, 2, 1.5)
# session['filtered_altitude_1d'] = [filtered_altitude.get_current_estimate(m) for m in session.enhanced_altitude]

filtered_altitude = KalmanFilter([session.enhanced_altitude[0], 0], 1, 2, 0.05)
session['filtered_altitude'] = [filtered_altitude.get_current_estimate(m) for m in session.enhanced_altitude]


# Rolling features
rolling_period = 15
# session['rolling_speed'] = session.speed.rolling(rolling_period).mean()
# session['rolling_power'] = session.power.rolling(rolling_period).mean()
session['rolling_altitude'] = session.enhanced_altitude.rolling(rolling_period).mean()

# plt.figure(figsize=(20, 8))
# # plt.plot(session.time, session.enhanced_altitude, label='enhanced_altitude')
# plt.plot(session.time, session.power, label='power')
# plt.plot(session.time, session.rolling_power, label='rolling_power')
# # plt.plot(session.time, session.enhanced_speed, label='enhanced_speed')
# # plt.plot(session.time, session.speed, label='speed')
# plt.legend()
# plt.show()

fig = go.Figure()

# fig.add_trace(go.Scatter(x=session.time, y=session.power, mode='lines', name='power'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_power, mode='lines', name='rolling power'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_power_1d, mode='lines', name='filtered power 1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_power, mode='lines', name='filtered power'))

fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_altitude, mode='lines', name='altitude'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_altitude, mode='lines', name='rolling altitude'))
# # fig.add_trace(go.Scatter(x=session.time, y=session.filtered_altitude_1d, mode='lines', name='filtered altitude 1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_altitude, mode='lines', name='filtered altitude'))

# fig.add_trace(go.Scatter(x=session.time, y=session.speed, mode='lines', name='speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_speed, mode='lines', name='rolling speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_speed, mode='lines', name='enhanced speed'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_speed1, mode='lines', name='filtered speed 1'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_speed2, mode='lines', name='filtered speed 2'))


fig.update_layout(width=1700, height=800)

fig.show()

## 2. Grade estimation

In [27]:
from custom_queue import Queue

def compute_grade_legacy(session, rolling_period):

    grade = []

    altitude = Queue(rolling_period+1)
    speed = Queue(rolling_period+1)

    for s in range(session.shape[0]):
        
        altitude.update(session.enhanced_altitude.values[s])
        speed.update(session.enhanced_speed.values[s])

        spd = speed.mean()
        n_seconds = altitude.count_not_null()
        dist = spd * n_seconds
        
        g = 100 * (altitude.current(2) - altitude.last(2)) / dist if ((n_seconds >= 5) & (dist > 1)) else 0
        g = 40 if (g > 40) else g
        g = -40 if (g < -40) else g

        grade.append(g)

    return grade

session['estimated_grade'] = compute_grade_legacy(session, 15)

In [28]:
filtered_grade = KalmanFilter1D(0, 1, 0.5, 0.1)
session['filtered_grade_1d'] = [filtered_grade.get_current_estimate(m) for m in session.estimated_grade]

# filtered_grade = KalmanFilter([0, 0], 1, 1.5, 0.05)
# session['filtered_grade'] = [filtered_grade.get_current_estimate(m) for m in session.estimated_grade]
# session['filtered_grade_spe'] = session.filtered_grade.map(lambda g: g**3 if (g > -1) & (g < 1) else g)

session['rolling_grade'] = session.estimated_grade.rolling(5).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(x=session.time, y=session.enhanced_altitude, mode='lines', name='altitude'))
fig.add_trace(go.Scatter(x=session.time, y=session.estimated_grade, mode='lines', name='estimated_grade'))

# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade_1d, mode='lines', name='filtered_grade_1d'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade, mode='lines', name='filtered_grade'))
# fig.add_trace(go.Scatter(x=session.time, y=session.filtered_grade_spe, mode='lines', name='filtered_grade_spe'))
# fig.add_trace(go.Scatter(x=session.time, y=session.rolling_grade, mode='lines', name='rolling_grade'))

fig.update_layout(width=1700, height=800)

fig.show()